# Data Elaboration Module
## Funzionamento
La classe *DataElaboration* contiene i metodi per l'elaborazione dei dati contenuti nel dataframe.
Il dataframe viene costruito partendo dai dati restituiti dalle TableApi di Cosmos DB (formato JSON)
## I metodi
Il metodo base è *DataFrame* che si occupa di creare il dataframe. Viene poi passato l'output al costruttore che lo rende disponibile agli altri metodi.
Tutti gli altri metodi effettuano delle elaborazioni filtrando i dati delle ultime 24h.
Nello specifico:
- *CalculateBloodPressureMedian* restituisce la media della pressione sanguigna
- *CalculateHeartBeatMedian* restituisce la media dei battiti cardiaci

In [8]:
import json
import warnings
import pandas as pd
import urllib.parse
from data import GetData
from decouple import config
from sqlalchemy import create_engine
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')

In [9]:
# Environment Variables
server = config("SERVER")
database = config("DATABASE") 
username = config("USERNAME")
password = config("PASSWORD")
driver = config("DRIVER")

In [21]:
# Create DB Connection with SQLAlchemy
odbc_str = 'DRIVER='+driver+';SERVER='+server+';PORT=1433;UID='+username+';DATABASE='+ database + ';PWD='+ password
connect_str = 'mssql+pyodbc:///?odbc_connect=' + urllib.parse.quote(odbc_str)
engine = create_engine(connect_str)

In [11]:
class DataElaboration():
    json = str()
    
    def __init__(self):
        self.json = self.GetDataFromAzure()
        self.data = self.CreateDataframe()
        self.bodytmp = self.CalculateBodyTemperatureMean()
        self.bldprs = self.CalculateBloodPressureMean()
        self.bldoxg = self.CalculateBloodOxygenMean()
        self.btr = self.CalculateBatteryMean()
        self.hrtfrq = self.CalculateHeartFrequencyMean()
        
    def CalculateMedians(self):
        return self.bodytmp, self.bldprs, self.bldoxg, self.btr, self.hrtfrq
        
    def GetDataFromAzure(self):
        d = GetData()
        return json.dumps(d.Get())
    
    def CreateDataframe(self):
        df = pd.read_json(self.json)
        return df
    
    def CalculateBodyTemperatureMean(self):
        df = self.data
        df[df['RowKey']>=str((datetime.now() - timedelta(days=1)))]['BodyTemperature']
        df = df[['BodyTemperature']].mean()
        return df.to_json()
    
    def CalculateBloodPressureMean(self):
        df = self.data
        df[df['RowKey']>=str((datetime.now() - timedelta(days=1)))]['BloodPressure']
        df = df[['BloodPressure']].mean()
        return df.to_json()
    
    def CalculateBloodOxygenMean(self):
        df = self.data
        df[df['RowKey']>=str((datetime.now() - timedelta(days=1)))]['BloodOxygen']
        df = df[['BloodOxygen']].mean()
        return df.to_json()
    
    def CalculateBatteryMean(self):
        df = self.data
        df[df['RowKey']>=str((datetime.now() - timedelta(days=1)))]['Battery']
        df = df[['Battery']].mean()
        return df.to_json()
    
    def CalculateHeartFrequencyMean(self):
        df = self.data
        df[df['RowKey']>=str((datetime.now() - timedelta(days=1)))]['HeartFrequency']
        df = df[['HeartFrequency']].mean()
        return df.to_json()

In [18]:
d = DataElaboration()
jdata = json.dumps(d.CalculateMedians())
try:
    engine.execute(f"INSERT INTO dbo.DeviceReports(ID, BodyTemperatureAvg, BloodPressureAvg, BloodOxygenAvg, BatteryAvg, HeartFrequencyAvg) VALUES {datetime.now() - timedelta(days=1)}, {jdata[0]}, {jdata[1]}, {jdata[2]}, {jdata[3]}")
except:
    raise BaseException "Cannot insert data into DB"

["{\"BodyTemperature\":36.4986762936}", "{\"BloodPressure\":109.5643802647}", "{\"BloodOxygen\":98.4933814681}", "{\"Battery\":19.0}", "{\"HeartFrequency\":54.4308062575}"]


InterfaceError: (pyodbc.InterfaceError) ('28000', "[28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user 'Devis'. (18456) (SQLDriverConnect); [28000] [Microsoft][ODBC Driver 17 for SQL Server]Invalid connection string attribute (0); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user 'Devis'. (18456); [28000] [Microsoft][ODBC Driver 17 for SQL Server]Invalid connection string attribute (0)")
(Background on this error at: https://sqlalche.me/e/14/rvf5)